In [ ]:
import numbers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

%config Completer.use_jedi = False


def get_mape(y_predict, y_true):
    return (abs(y_predict - y_true) / y_true).mean()


def process_rooms_number(x):
        
    if pd.isna(x):
        return 1
    
    if isinstance(x, int):
        return x
    
    if x.isdigit():
        return int(x)
    
    if x == 'Студия':
        return 1
    
    if x == 'Своб. планировка':
        return 1
    
    if x == '> 9':
        return 10

    return 1

<h3>Реализуем линейную регрессию</h3>

<p>Чаще всего алгоритмы машинного обучения реализуются в виде классов с обязательными методами <code>.fit()</code>, <code>.predict()</code>. </p>

<p><code>.fit()</code> – обучить алгоритм на обучающей выборке;</p>

<p><code>.predict()</code> – сделать предсказание на тестовых данных.</p>

<p> </p>

In [ ]:
class LinearRegression:
    
    def __init__(self, max_iter=1e4, lr=0.001, tol=0.001, print_every=100):
        
        self.max_iter = max_iter
        self.lr = lr
        self.tol = tol
        self.print_every = print_every
        
        self.weights = None
        self.bias = None
        
    def fit(self, X_train, y_train, X_val, y_val):
        
        self.check_regression_X_y(X_train, y_train)
        self.check_regression_X_y(X_val, y_val)
        
        n, m = X_train.shape
        
        self.weights = np.zeros((m, 1))
        self.bias = np.median(y_train)
        
        n_iter = 0
        gradient_norm = np.inf
        
        while n_iter < self.max_iter and gradient_norm > self.tol:
            

            dJdw, dJdb = self.grads(X_train, y_train)
                
            gradient_norm = np.linalg.norm(np.hstack([dJdw.flatten(), [dJdb]]))
                
            self.weights = self.weights - self.lr * dJdw
            self.bias = self.bias - self.lr * dJdb
            
            n_iter += 1
            
            if n_iter % self.print_every == 0:
                self.print_metrics(X_train, y_train, X_val, y_val, n_iter, gradient_norm)
        
        return self

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias
    
    def grads(self, X, y):
        
        y_hat = self.predict(X)
        
        dJdw = 2 * (X.T @ (y_hat - y)) / len(X)
        dJdb = 2 * (y_hat - y).mean()
        
        self.check_grads(dJdw, dJdb)
        
        return dJdw, dJdb
    
    def print_metrics(self, X_train, y_train, X_val, y_val, n_iter, gradient_norm):
        
        train_preds = self.predict(X_train)
        val_preds = self.predict(X_val)
        
        MAPE_train = get_mape(train_preds, y_train)
        MAPE_val = get_mape(val_preds, y_val)
        MSE_train = ((y_train - self.predict(X_train)) ** 2).mean()
        
        print(f'{n_iter} completed. MAPE on train: {round(MAPE_train, 3)}, val: {round(MAPE_val, 3)},  grad norm: {gradient_norm}, MSE: {round(MSE_train, 2)}')
        
        
    def check_grads(self, dJdw, dJdb):
        
        if not isinstance(dJdb, numbers.Real):
            raise ValueError(f'Производная по параметру b должна быть действительным '
                             f'числом, как и сам параметр b, а у нас {dJdb} типа {type(dJdb)}')
            
        if dJdw.shape != self.weights.shape:
            raise ValueError(f'Размерность градиента по параметрам w должна совпадать с самим вектором w, '
                             f'а у нас dJdw.shape = {dJdw.shape} не совпадает с weight.shape = {self.weights.shape}')
            
        
    @staticmethod
    def check_regression_X_y(X, y):
        
        if X.shape[0] == 0:
            raise ValueError(f'X и y не должны быть пустыми, а у нас X.shape = {X.shape} и y.shape = {y.shape}')
            
      #  if np.isnan(X).any():
      #      raise ValueError(f'X не должен содержать "not a number" (np.nan)')
            
      #  if np.isnan(y).any():
      #      raise ValueError(f'y не должен содержать "not a number" (np.nan)')
        
        if X.shape[0] != y.shape[0]:
            raise ValueError(f'Длина X и y должна быть одинаковой, а у нас X.shape = {X.shape}, y.shape = {y.shape}')
            
        if y.shape[1] != 1:
            raise ValueError(f'y - вектор ответов должен быть размерности (m, 1), а у нас y.shape = {y.shape}')
                    
        if np.any([(not isinstance(value, numbers.Real)) for value in y.flatten()]):
            raise ValueError(f'{y.flatten()[0]}Ответы на объектах должны быть действительными числами!')
       

<h3>Тестируем модель на простой задаче</h3>

In [ ]:
X = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
    [1, 1, 1],
])
y = np.array([[1], [2], [3], [4]])
model = LinearRegression(lr=0.1)
model.fit(X, y, X, y)
model.predict(X)

100 completed. MAPE on train: 0.027, val: 0.027,  grad norm: 0.04184749716482497, MSE: 0.0
200 completed. MAPE on train: 0.002, val: 0.002,  grad norm: 0.002752354006169089, MSE: 0.0


array([[1.0010693 ],
       [2.00106431],
       [3.00105932],
       [3.99815656]])

<h3>Решаем задачу предсказания цены</h3>

In [ ]:
data = pd.read_csv('/binary_clf_data (1).csv')

In [ ]:
data

,gender,user_id,category_id,category_name,subcategory_id,subcategory_name,param1,param2,param3,param1_microcat_id,param2_microcat_id,param3_microcat_id
0,male,165823598,1,Для дома и дачи,50,Ремонт и строительство,Стройматериалы,NaN,NaN,981.0,NaN,NaN
1,male,165823598,1,Для дома и дачи,50,Ремонт и строительство,Стройматериалы,NaN,NaN,981.0,NaN,NaN
2,male,165823598,28,Личные вещи,22,"Одежда, обувь, аксессуары",Мужская одежда,Обувь,45,3285.0,6813.0,6074.0
3,male,165823598,1,Для дома и дачи,38,Мебель и интерьер,Шкафы и комоды,NaN,NaN,4533.0,NaN,NaN
4,male,154189396,1,Для дома и дачи,38,Мебель и интерьер,Кухонные гарнитуры,NaN,NaN,7506.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
8918,female,154265680,250002,Услуги,250005,Предложение услуг,"Праздники, мероприятия",NaN,NaN,3705.0,NaN,NaN
8919,male,154266216,1,Для дома и дачи,50,Ремонт и строительство,Камины и обогреватели,NaN,NaN,5181.0,NaN,NaN
8920,male,154266216,1,Для дома и дачи,50,Ремонт и строительство,Камины и обогреватели,NaN,NaN,5181.0,NaN,NaN
8921,male,154268351,250003,Работа,250004,Резюме,"Без опыта, студенты",NaN,NaN,2179.0,NaN,NaN


In [ ]:
data.describe()

,user_id,category_id,subcategory_id,param1_microcat_id,param2_microcat_id,param3_microcat_id
count,8.923000e+03,8923.000000,8923.000000,8.570000e+03,3.395000e+03,2.119000e+03
mean,1.542326e+08,29856.645747,30821.021069,1.441218e+08,1.593770e+09,3.266854e+08
std,2.464603e+05,81050.172850,83588.724863,4.707791e+09,1.123054e+10,7.187231e+09
min,1.541894e+08,1.000000,2.000000,1.500000e+01,5.000000e+01,3.800000e+01
25%,1.542092e+08,12.000000,22.000000,1.989000e+03,2.748000e+03,2.236500e+03
50%,1.542277e+08,28.000000,38.000000,3.028000e+03,5.483000e+03,4.522000e+03
75%,1.542457e+08,28.000000,50.000000,5.144000e+03,6.949000e+03,6.708000e+03
max,1.658236e+08,250003.000000,500001.000000,1.541490e+11,8.112375e+10,1.648015e+11


<p>Чистим данные:</p>

In [ ]:
data = data.drop_duplicates(subset=['item_id'], keep='last')
data = data.dropna(subset=['area'])
data['rooms_number'] = data['rooms_number'].apply(process_rooms_number).copy()
data = data[(data.price > 970000) & (data.price < 12700000)]
data = data[(data.floor < 59)]
data = data[(data.floors_in_house < 59)]

data = data.dropna(axis=0)

In [ ]:
data = data.fillna(0)
data = data.replace({'female': 1, 'male':0})

In [ ]:
train, val, train_price, val_price = train_test_split(data.drop('gender', axis=1), data['gender'], random_state=42)

In [ ]:
train_price

2061    0
549     0
6850    1
7474    1
7826    0
       ..
5734    1
5191    1
5390    1
860     1
7270    0
Name: gender, Length: 6692, dtype: int64

In [ ]:
test_data = pd.read_csv('/binary_clf_data (1).csv')
test_data = test_data.fillna(0)
test_data = test_data.replace({'female': 1, 'male':0})
test, test_price = test_data.drop('gender', axis=1), test_data['gender']

y_train = train_price.values.reshape(-1, 1)
y_val = val_price.values.reshape(-1, 1)

In [ ]:
test_data.head()

,gender,user_id,category_id,category_name,subcategory_id,subcategory_name,param1,param2,param3,param1_microcat_id,param2_microcat_id,param3_microcat_id
0,0,165823598,1,Для дома и дачи,50,Ремонт и строительство,Стройматериалы,0,0,981.0,0.0,0.0
1,0,165823598,1,Для дома и дачи,50,Ремонт и строительство,Стройматериалы,0,0,981.0,0.0,0.0
2,0,165823598,28,Личные вещи,22,"Одежда, обувь, аксессуары",Мужская одежда,Обувь,45,3285.0,6813.0,6074.0
3,0,165823598,1,Для дома и дачи,38,Мебель и интерьер,Шкафы и комоды,0,0,4533.0,0.0,0.0
4,0,154189396,1,Для дома и дачи,38,Мебель и интерьер,Кухонные гарнитуры,0,0,7506.0,0.0,0.0


### Делаем бейзлайн

In [ ]:
get_mape(y_predict=np.median(y_train), y_true=y_val)

TypeError: ignored

### Обучаем модель

In [ ]:
y_train

array([['male'],
       ['male'],
       ['female'],
       ...,
       ['female'],
       ['female'],
       ['male']], dtype=object)

1) Начинаем с простого

In [ ]:
X_train = train[['category_id']].values
X_val = val[['category_id']].values


model = LinearRegression(lr=1e-4, max_iter=14000, print_every=10000, tol=0.1)
model.fit(X_train, y_train, X_val, y_val)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: RuntimeWarning: overflow encountered in matmul
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in subtract


<p>Для того, чтобы начать ориентироваться в метрике решения задачи, очень важно построить одну или несколько простых моделей. Часто есть соблазн добавить все признаки сразу и обучить модель — мы так поступать не будем. Наоборот, мы будем постепенно добавлять признаки и следить за тем, что модель решает задачу лучше и лучше. </p>

2) Увеличиваем количество признаков

In [ ]:
X_train = train[['category_id', 'subcategory_id']].values
X_val = val[['category_id', 'subcategory_id']].values

model = LinearRegression(lr=1e-4, max_iter=120000, print_every=10000, tol=0.1)
model.fit(X_train, y_train, X_val, y_val)

TypeError: ignored

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
train=pd.read_csv('/binary_clf_data (1).csv')
train.head()

,gender,user_id,category_id,category_name,subcategory_id,subcategory_name,param1,param2,param3,param1_microcat_id,param2_microcat_id,param3_microcat_id
0,male,165823598,1,Для дома и дачи,50,Ремонт и строительство,Стройматериалы,NaN,NaN,981.0,NaN,NaN
1,male,165823598,1,Для дома и дачи,50,Ремонт и строительство,Стройматериалы,NaN,NaN,981.0,NaN,NaN
2,male,165823598,28,Личные вещи,22,"Одежда, обувь, аксессуары",Мужская одежда,Обувь,45,3285.0,6813.0,6074.0
3,male,165823598,1,Для дома и дачи,38,Мебель и интерьер,Шкафы и комоды,NaN,NaN,4533.0,NaN,NaN
4,male,154189396,1,Для дома и дачи,38,Мебель и интерьер,Кухонные гарнитуры,NaN,NaN,7506.0,NaN,NaN


In [ ]:
#train = train.dropna()
train = train.fillna(-1)
train = train.replace({'female': 1, 'male':0})
#train.drop(['category_name', 'subcategory_name', 'param1', 'param2', 'param3'], axis=1, inplace=True)
train.head()

,gender,user_id,category_id,category_name,subcategory_id,subcategory_name,param1,param2,param3,param1_microcat_id,param2_microcat_id,param3_microcat_id
0,0,165823598,1,Для дома и дачи,50,Ремонт и строительство,Стройматериалы,-1,-1,981.0,-1.0,-1.0
1,0,165823598,1,Для дома и дачи,50,Ремонт и строительство,Стройматериалы,-1,-1,981.0,-1.0,-1.0
2,0,165823598,28,Личные вещи,22,"Одежда, обувь, аксессуары",Мужская одежда,Обувь,45,3285.0,6813.0,6074.0
3,0,165823598,1,Для дома и дачи,38,Мебель и интерьер,Шкафы и комоды,-1,-1,4533.0,-1.0,-1.0
4,0,154189396,1,Для дома и дачи,38,Мебель и интерьер,Кухонные гарнитуры,-1,-1,7506.0,-1.0,-1.0


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train.drop('gender',axis=1), train['gender'], test_size=0.30, random_state=101)
                    

In [ ]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
print(classification_report(y_test,predictions))
val_acc = accuracy_score(predictions, y_test)
print(val_acc)

              precision    recall  f1-score   support

           0       0.85      0.23      0.36       221
           1       0.70      0.98      0.82       415

    accuracy                           0.72       636
   macro avg       0.78      0.60      0.59       636
weighted avg       0.76      0.72      0.66       636

0.7185534591194969


In [ ]:
zadane=pd.read_csv('/dataset_527992_9 (2).txt')
zadane.head()

,user_id,category_id,category_name,subcategory_id,subcategory_name,param1,param2,param3,param1_microcat_id,param2_microcat_id,param3_microcat_id
0,154189609,7,Хобби и отдых,4,Велосипеды,Детские,NaN,NaN,1246.0,NaN,NaN
1,154189609,17,Бытовая электроника,29,Товары для компьютера,Комплектующие,Видеокарты,NaN,1651.0,6491.0,NaN
2,154189609,17,Бытовая электроника,41,Телефоны,iPhone,NaN,NaN,7315.0,NaN,NaN
3,154189609,17,Бытовая электроника,41,Телефоны,Xiaomi,NaN,NaN,10750009.0,NaN,NaN
4,154189728,28,Личные вещи,22,"Одежда, обувь, аксессуары",Женская одежда,Верхняя одежда,46–48 (L),1989.0,6949.0,775.0


In [ ]:
zadane = zadane.dropna()
#train = train.fillna(-1)
zadane = zadane.replace({'female': 1, 'male':0})
zadane.drop(['category_name', 'subcategory_name', 'param1', 'param2', 'param3'], axis=1, inplace=True)
zadane.head()

,user_id,category_id,subcategory_id,param1_microcat_id,param2_microcat_id,param3_microcat_id
4,154189728,28,22,1989.0,6949.0,775.0
5,154189879,28,22,3285.0,6813.0,5386.0
6,154189879,28,22,1989.0,2679.0,3980.0
7,154189879,28,22,1989.0,2679.0,3980.0
8,154189879,28,22,1989.0,2679.0,6644.0


In [ ]:
predictions = logmodel.predict(zadane)

In [ ]:
predictions

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
predictions_test = pd.DataFrame()
predictions_test["user_id"] = zadane.user_id
predictions_test["gender"] = predictions
predictions_test["gender"] = predictions_test["gender"].replace({1:'female', 0:'male'})
predictions_test

,user_id,gender
4,154189728,female
5,154189879,female
6,154189879,female
7,154189879,female
8,154189879,female
...,...,...
3166,154271494,female
3167,154271494,female
3168,154271494,female
3169,154271494,female


In [ ]:
predictions_test.to_csv('test_predictions.csv', index=False)


In [ ]:
from google.colab import files
files.download('test_predictions.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Делаем новые признаки
##### One hot encoding

In [ ]:
ohe_example = pd.DataFrame({'feature': ['a', 'b', 'a', 'c']})
ohe_example

,feature
0,a
1,b
2,a
3,c


In [ ]:
ohe = OneHotEncoder(sparse=False)
ohe.fit_transform(ohe_example)

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [ ]:
train.district.value_counts()

Октябрьский        5335
Кировский          4801
Ленинский          4381
Калининский        2713
Заельцовский       2710
Дзержинский        2104
Центральный        1702
Первомайский       1590
Советский          1346
Железнодорожный    1039
Name: district, dtype: int64

In [ ]:
ohe_house_type_transformer = OneHotEncoder(sparse=False)
train_ohe_house_type = ohe_house_type_transformer.fit_transform(train[['type_of_house']])
val_ohe_house_type = ohe_house_type_transformer.transform(val[['type_of_house']])

ohe_district_transformer = OneHotEncoder(sparse=False)
train_ohe_district = ohe_district_transformer.fit_transform(train[['district']])
val_ohe_district = ohe_district_transformer.transform(val[['district']])
print(train_ohe_district)

X_train_extended = np.hstack([X_train, train_ohe_house_type, train_ohe_district])
X_val_extended = np.hstack([X_val, val_ohe_house_type, val_ohe_district])

model = LinearRegression(lr=1e-4, max_iter=120000, print_every=10000, tol=0.1)
model.fit(X_train_extended, y_train, X_val_extended, y_val)

KeyError: ignored

### Задание на семинаре: попробовать улучшить метрику MAPE до 15.8% (топ-1 без ML с первой недели).

Варианты путей для улучшения:

    1) Делать новые признаки из существующих;
    2) Препроцессинг данных, целевой переменной - постпроцессинг ответов модели;
    3) Анализ ошибок модели –> генерация идей;
    4) Добавить регуляризацию;


<h3>Задание на семинаре: реализуем логистическую регрессию</h3>

<p>Мы получаем оптимальные веса алгоритма градиентным спуском:</p>

<p style="text-align:center"><br />
<br />
<span class="math-tex">\(\begin{bmatrix} w_{1}^{t+1}\\  ...\\ w_{m}^{t+1}\\  \end{bmatrix} = \begin{bmatrix} w_{1}^{t}\\  ...\\ w_{m}^{t}\\  \end{bmatrix} - \alpha \cdot  \begin{bmatrix} \sum_{i=1}^{n} (\frac{1}{1+exp(w^{T}x^{(i)})} - y^{(i)})x_{1}^{(i)}\\  ...\\ \sum_{i=1}^{n} (\frac{1}{1+exp(w^{T}x^{(i)})} - y^{(i)})x_{m}^{(i)}\\  \end{bmatrix}\)</span></p>

<p style="text-align:center"><span class="math-tex">\(b^{t+1} = b^{t} - \alpha \sum_{i=1}^{n} (\frac{1}{1+exp(w^{T}x^{(i)})} - y^{(i)})\)</span></p>

<p style="text-align:center">&nbsp;</p>

<p>&nbsp;</p>

In [ ]:
from sklearn.metrics import accuracy_score


class LogisticRegression:
    
    def __init__(self, max_iter=1e4, lr=0.001, tol=0.001, print_every=100):
        
        '''
        max_iter – максимальное количеств
        '''
        
        self.max_iter = max_iter
        self.lr = lr
        self.tol = tol
        self.print_every = print_every
        
        self.weights = None
        self.bias = None
        
    def fit(self, X_train, y_train, X_val, y_val):
        
        '''
        Обучение модели.
        
        X_train – матрица объектов для обучения
        y_train – ответы на объектах для обучения
        
        X_val – матрица объектов для валидации
        y_val – ответы на объектах для валидации
        '''
        
        self.check_binary_clf_X_y(X_train, y_train)
        self.check_binary_clf_X_y(X_val, y_val)
                
        n, m = X_train.shape
        
        self.weights = 
        self.bias = 
        
        n_iter = 0
        gradient_norm = np.inf
        
        while n_iter < self.max_iter and gradient_norm > self.tol:
            
            dJdw, dJdb = self.grads(X_train, y_train)
            gradient_norm = np.linalg.norm(np.hstack([dJdw.flatten(), [dJdb]]))
                
            self.weights = 
            self.bias = 
            
            n_iter += 1
            
            if n_iter % self.print_every == 0:
                self.print_metrics(X_train, y_train, X_val, y_val, n_iter, gradient_norm)
        
        return self
    
    def predict(self, X):  
        
        '''
        Метод возвращает предсказанную метку класса на объектах X
        '''
        
        pass
        
    
    def predict_proba(self, X):
        
        '''
        Метод возвращает вероятность класса 1 на объектах X
        '''
        pass
    
    def grads(self, x, y):
        
        '''
        Рассчёт градиентов
        '''
        y_hat = 
        
        dJdw = 
        dJdb = 
        
        self.check_grads(dJdw, dJdb)
        
        return dJdw, dJdb
    
    @staticmethod
    def sigmoid(x):
        '''
        Сигмоида от x
        '''
        pass
    
    def print_metrics(self, X_train, y_train, X_val, y_val, n_iter, gradient_norm):
        
        train_preds = self.predict(X_train)
        val_preds = self.predict(X_val)
        
        train_acc = accuracy_score(train_preds, y_train)
        val_acc = accuracy_score(val_preds, y_val)
        
        print(f'{n_iter} completed. accuracy_score on train: {train_acc}, val: {val_acc}, grad_norm: {gradient_norm}')
        
    def check_grads(self, dJdw, dJdb):
        
        if not isinstance(dJdb, numbers.Real):
            raise ValueError(f'Производная по параметру b должна быть действительным'
                             f' числом, как и сам параметр b, а у нас {dJdb} типа {type(dJdb)}')
            
        if dJdw.shape != self.weights.shape:
            raise ValueError(f'Размерность градиента по параметрам w должна совпадать с самим вектором w, '
                             f'а у нас dJdw.shape = {dJdw.shape} не совпадает с weight.shape = {self.weights.shape}')
    
    @staticmethod
    def check_binary_clf_X_y(X, y):
        
        if X.shape[0] == 0:
            raise ValueError(f'X и y не должны быть пустыми, а у нас X.shape = {X.shape} и y.shape = {y.shape}')
            
        if np.isnan(X).any():
            raise ValueError(f'X не должен содержать "not a number" (np.nan)')
            
        if np.isnan(y).any():
            raise ValueError(f'y не должен содержать "not a number" (np.nan)')
        
        if X.shape[0] != y.shape[0]:
            raise ValueError(f'Длина X и y должна быть одинаковой, а у нас X.shape = {X.shape}, y.shape = {y.shape}')
            
        if y.shape[1] != 1:
            raise ValueError(f'y - вектор ответов должен быть размерности (m, 1), а у нас y.shape = {y.shape}')

                    
        if sorted(np.unique([1, 0, 0])) != [0, 1]:
            raise ValueError(f'Ответы на объектах должны быть только 0 или 1, а у нас np.unique(y) = {np.unique(y)}')


<h2>Домашнее задание</h2>

<p>Воспользуемся реализованной моделью логистической регрессии, чтобы решить задачу определения пола пользователя Авито.</p>

<p><a href="https://stepik.org/media/attachments/lesson/527992/binary_clf_data.csv" rel="noopener noreferrer nofollow">Данные</a> даны в сыром виде &ndash; айтемы и их категории, которые выкладывали покупатели на Авито. Целевая переменная: <em>gender.</em></p>

<p>Вам необходимо разбить данные на train, val. Перед загрузкой файла с ответом убедитесь, что точность (<a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html" rel="noopener noreferrer nofollow">accuracy</a>)&nbsp;на валидации не менее 0.7.</p>

<p>&nbsp;</p>

<p><strong>План действий</strong></p>

<p>Сначала нужно преобразовать категории с помощью one-hot encoding. Далее необходимо агрегировать категории, в которых пользователи выкладывали объявления, чтобы получить вектор признаков для каждого объекта. В результате у каждого пользователя будет вектор признаков, содержащий количество айтемов, выложенных в каждой из возможных категорий.</p>

<ul>
	<li>Убедитесь, что для каждого пользователя в выборке есть только один объект, каждый признак означает количество айтемов, выложенное этим пользователем в категории;</li>
	<li>Убедитесь, что после one-hot энкодинга каждая категория соответствует признаку,&nbsp;<strong>одинаковому в train, val и test.</strong></li>
</ul>

<p>Попробуйте варианты отбора признаков. Для борьбы с переобучением на редких категориях используйте регуляризацию. В качестве&nbsp;ответа загрузите файл с предсказанием пола для пользователей:</p>

<p style="text-align:center">&nbsp;</p>

<table align="center" border="1" cellpadding="1" cellspacing="1" style="width:500px">
	<thead>
		<tr>
			<th style="text-align:center">user_id</th>
			<th style="text-align:center">gender</th>
		</tr>
	</thead>
	<tbody>
		<tr>
			<td style="text-align:center">15424171</td>
			<td style="text-align:center">male</td>
		</tr>
		<tr>
			<td style="text-align:center">15454025</td>
			<td style="text-align:center">female</td>
		</tr>
	</tbody>
</table>

<p style="text-align:center">&nbsp;</p>

<p>Такой файл можно сформировать с помощью&nbsp;<code>test_predictions.to_csv(&#39;test_predictions.csv&#39;, index=False)</code>.</p>

<p>После того, как получилось обучить модель, ответьте на вопрос: какие из категорий вносят наибольший вклад в вероятность класса &quot;мужчина&quot; и класса &quot;женщина&quot;?</p>

<p>Например, если вы закодировали &quot;мужчина&quot; как 1, большие положительные веса при признаках будут означать большой вклад в вероятность класса 1, большие по модулю отрицательные веса будут вносить наибольший вклад в вероятность класса 0. Согласуется ли полученный результат с вашим жизненным опытом?</p>
